In [8]:
import json
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

from lib.model import CLIPModel
from lib import utils, model, dataset, training, codes

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
class CFG:
    def __init__(self, cfg):
        for key, val in cfg.items():
            self.__dict__[key] = val

def open_cfg(file):
    with open(file, 'rt') as f:
        data = json.loads(f.read())


    cfg = CFG(data)
    return cfg
    
def load_model(config_path, weights_path):
    config = open_cfg(config_path)
    net = CLIPModel(config)
    net.load_state_dict(torch.load(weights_path, weights_only=True))
    net.to(config.device)


    utils.set_seed(config.seed)
    df = utils.get_data_cached(config.data_path, codes.DECODE_DICT, config.cache_path + '/df.csv')
    train, test = train_test_split(df, test_size=config.test_size, random_state=config.seed)
    test_ds = dataset.CLIP_ECG_Dataset(test, config)
    test_dl = torch.utils.data.DataLoader(test_ds, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=True)
    metrics = training.valid_epoch(net, test_dl, config.test_classes, config) 
    config.test_metrics = metrics

    cfg = {k:v for k, v in config.__dict__.items() if not k.startswith('__')}
    with open(config_path, 'w') as fp:
        json.dump(cfg, fp)
    
    return metrics

results = load_model('results/0015de3c6198.cfg', 'results/0015de3c6198.pt')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:07<00:00,  2.41it/s]


In [3]:
best_configs = ['7a0ff6b02309', 'c99398b9f189', 'd34730a0aeca']

In [6]:
metrics = list()
for cfg in best_configs:
    metrics.append(load_model(f'results/{cfg}.cfg', f'results/{cfg}.pt'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.82it/s]


In [15]:
{key: val for key, val in pd.DataFrame(metrics).mean().to_dict().items() if key.endswith('_rocauc')}

{'abnormal QRS_rocauc': 0.4058183411490865,
 'atrial fibrillation_rocauc': 0.8669222449333563,
 'incomplete right bundle branch block_rocauc': 0.8585417279571983,
 'left anterior fascicular block_rocauc': 0.9363336735159421,
 'left axis deviation_rocauc': 0.7547334357901834,
 'left ventricular hypertrophy_rocauc': 0.7268959485168128,
 'myocardial infarction_rocauc': 0.8053573071097303,
 'myocardial ischemia_rocauc': 0.8599167877523041,
 'sinus rhythm_rocauc': 0.8462469537123315,
 'st depression_rocauc': 0.6289211109359859,
 't wave abnormal_rocauc': 0.8477740452270924,
 'ventricular ectopics_rocauc': 0.8972463667943754}